In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder


In [2]:
df = pd.read_csv('data.csv')

C:\Users\valer\AppData\Local\Temp\ipykernel_14132\1507164070.py:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552912 entries, 0 to 3552911
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   maker                object 
 1   model                object 
 2   mileage              float64
 3   manufacture_year     float64
 4   engine_displacement  float64
 5   engine_power         float64
 6   body_type            object 
 7   color_slug           object 
 8   stk_year             float64
 9   transmission         object 
 10  door_count           float64
 11  seat_count           float64
 12  fuel_type            object 
 13  date_created         object 
 14  date_last_seen       object 
 15  price_eur            float64
dtypes: float64(8), object(8)
memory usage: 433.7+ MB


In [4]:
df = df[(df['manufacture_year'] >= 1886) & (df['manufacture_year'] <= 2025)]


In [5]:
null_percentage = df.isnull().mean() * 100
null_percentage

maker                  13.952268
model                  31.556399
mileage                 7.155385
manufacture_year        0.000000
engine_displacement    18.565621
engine_power           12.862821
body_type              33.278774
color_slug             93.430615
stk_year               84.401179
transmission           18.564414
door_count             27.716251
seat_count             32.877884
fuel_type              51.527582
date_created            0.000000
date_last_seen          0.000000
price_eur               0.000000
dtype: float64

In [6]:
#columns_to_drop = null_percentage[null_percentage > 60].index
#df_cleaned = df.drop(columns=columns_to_drop)

In [7]:
#df_cleaned = df_cleaned.dropna()

In [8]:
#df = df_cleaned

In [9]:
# df = df.dropna()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3149488 entries, 0 to 3552910
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   maker                object 
 1   model                object 
 2   mileage              float64
 3   manufacture_year     float64
 4   engine_displacement  float64
 5   engine_power         float64
 6   body_type            object 
 7   color_slug           object 
 8   stk_year             float64
 9   transmission         object 
 10  door_count           float64
 11  seat_count           float64
 12  fuel_type            object 
 13  date_created         object 
 14  date_last_seen       object 
 15  price_eur            float64
dtypes: float64(8), object(8)
memory usage: 408.5+ MB


In [ ]:
df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
df['date_last_seen'] = pd.to_datetime(df['date_last_seen'], errors='coerce')


df = df.dropna(subset=['date_created', 'date_last_seen'], how='all')


df['days_since_created'] = (df['date_last_seen'] - df['date_created']).dt.days
df.drop(['date_created', 'date_last_seen'], axis=1, inplace=True)


current_year = pd.Timestamp.now().year
df['car_age'] = current_year - df['manufacture_year']


luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi']
df['is_luxury_brand'] = df['maker'].isin(luxury_brands).astype(int)


df['mileage_per_year'] = df['mileage'] / df['car_age'].replace(0, np.nan)


df['age_bin'] = pd.cut(df['car_age'], bins=[-1, 2, 5, 100], labels=['new', 'mid_age', 'old'])


categorical_cols = ['maker', 'model', 'body_type', 'transmission', 'fuel_type', 'color_slug', 'age_bin']
numerical_cols = ['mileage', 'manufacture_year', 'engine_displacement', 
                  'engine_power', 'stk_year', 'door_count', 'seat_count',
                  'days_since_created', 'car_age', 'mileage_per_year']


imputer = IterativeImputer(max_iter=10, random_state=42)
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])


iso = IsolationForest(contamination=0.005, random_state=42)
outliers = iso.fit_predict(df[numerical_cols])
df = df[outliers != -1].reset_index(drop=True)


df.dropna(inplace=True)


X = df.drop('price_eur', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


target_encoder = ce.TargetEncoder(cols=categorical_cols)
X_train[categorical_cols] = target_encoder.fit_transform(X_train[categorical_cols], y_train)
X_test[categorical_cols] = target_encoder.transform(X_test[categorical_cols])

scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


print("✅ Final dataset size:", df.shape)


c:\Users\valer\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


✅ Final dataset size: (91504, 19)


In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
   
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    percentage_error = abs((y_pred - y_test) / y_test) * 100
    threshold = 10
    accuracy = (percentage_error < threshold).mean() * 100

    return mae, mse, r2, accuracy

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91504 entries, 939273 to 3131734
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   maker                91504 non-null  object  
 1   model                91504 non-null  object  
 2   mileage              91504 non-null  float64 
 3   manufacture_year     91504 non-null  float64 
 4   engine_displacement  91504 non-null  float64 
 5   engine_power         91504 non-null  float64 
 6   body_type            91504 non-null  object  
 7   color_slug           91504 non-null  object  
 8   stk_year             91504 non-null  float64 
 9   transmission         91504 non-null  object  
 10  door_count           91504 non-null  float64 
 11  seat_count           91504 non-null  float64 
 12  fuel_type            91504 non-null  object  
 13  price_eur            91504 non-null  float64 
 14  days_since_created   91504 non-null  float64 
 15  car_age          

In [17]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=8, random_state=42)
evaluate_model(xgb, X_train, X_test, y_train, y_test)

mae, mse, r2, accuracy = evaluate_model(xgb, X_train, X_test, y_train, y_test)
print(f"XGBRegressor: Accuracy: {accuracy}%, MAE: {mae}, MSE: {mse}, R²: {r2}")


XGBRegressor: Accuracy: 87.21404633542183%, MAE: 324.92093528443513, MSE: 26806764.686830103, R²: 0.07503047401961493


In [18]:
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
mae, mse, r2, accuracy = evaluate_model(random_forest, X_train, X_test, y_train, y_test)

print(f"RandomForest: Accuracy: {accuracy}%, MAE: {mae}, MSE: {mse}, R²: {r2}")

RandomForest: Accuracy: 91.41046189712954%, MAE: 322.5232892604207, MSE: 33628950.77265283, R²: -0.1603695939734322


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91504 entries, 939273 to 3131734
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   maker                91504 non-null  object  
 1   model                91504 non-null  object  
 2   mileage              91504 non-null  float64 
 3   manufacture_year     91504 non-null  float64 
 4   engine_displacement  91504 non-null  float64 
 5   engine_power         91504 non-null  float64 
 6   body_type            91504 non-null  object  
 7   color_slug           91504 non-null  object  
 8   stk_year             91504 non-null  float64 
 9   transmission         91504 non-null  object  
 10  door_count           91504 non-null  float64 
 11  seat_count           91504 non-null  float64 
 12  fuel_type            91504 non-null  object  
 13  price_eur            91504 non-null  float64 
 14  days_since_created   91504 non-null  float64 
 15  car_age          